## Surgical Tool Detection

###### Identify all surgical tools that are present in an image

<img src="tool_name.jpg">

In [3]:
# load the packages
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from glob import glob
from tqdm import tqdm
import h5py

%matplotlib inline

### Read Video files

In [7]:
def video2frame(filepath, output):
    #path = filepath+filename+'.mp4';
    filename = os.path.basename(filepath).split('.')[0]    
    cap = cv2.VideoCapture(filepath)
    if cap.isOpened() is False:
        print('Error opening video file '+str(filepath))
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if frame_count%25 == 0:
            if ret is True:
                frame_name = filename + '_' + str(frame_count)
                frame_path = os.path.join(output,frame_name + '.' + 'jpg')
                cv2.imwrite(frame_path,frame)
            else:
                break
        frame_count+=1
    print(filepath, filename, frame_count/25)

## Extract training data 

In [6]:
filepath = 'data/m2cai16-tool/train_dataset/*.mp4'
output_path = 'data/m2cai16-tool/train_dataset/images'

# Clean dir
if not os.path.exists(output_path):
    os.mkdir(output_path)
for filepath in glob(output_path+'/*'):
    os.remove(filepath)
    
for filename in glob(filepath):
    video2frame(filename, output_path)
    

data/m2cai16-tool/train_dataset\tool_video_01.mp4 tool_video_01 2354.0
data/m2cai16-tool/train_dataset\tool_video_02.mp4 tool_video_02 1973.0
data/m2cai16-tool/train_dataset\tool_video_03.mp4 tool_video_03 4576.0
data/m2cai16-tool/train_dataset\tool_video_04.mp4 tool_video_04 1195.0
data/m2cai16-tool/train_dataset\tool_video_05.mp4 tool_video_05 2516.0
data/m2cai16-tool/train_dataset\tool_video_06.mp4 tool_video_06 3107.0
data/m2cai16-tool/train_dataset\tool_video_07.mp4 tool_video_07 1357.0
data/m2cai16-tool/train_dataset\tool_video_08.mp4 tool_video_08 1635.0
data/m2cai16-tool/train_dataset\tool_video_09.mp4 tool_video_09 1924.0
data/m2cai16-tool/train_dataset\tool_video_10.mp4 tool_video_10 2650.0


## Read text files

In [51]:
train_dfs = []
train_label_path = 'data/m2cai16-tool/train_dataset/tool_video_??.txt'
# train_label_path = 'data/m2cai16-tool/train_dataset/tool_video_01.txt'
for filepath in glob(train_label_path):    
    tdf = pd.read_table(filepath)
    train_dfs.append(tdf)    
train_dfs = pd.concat(train_dfs)
train_dfs.drop('Frame', 1, inplace=True)
train_dfs.head()

,Grasper,Bipolar,Hook,Scissors,Clipper,Irrigator,SpecimenBag
0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0


In [53]:
train_dfs = np.array(train_dfs)
train_dfs[76]

array([1, 0, 0, 0, 0, 0, 0], dtype=int64)

## Load Images 

In [2]:
train_images_pattern = 'data/m2cai16-tool/train_dataset/images/tool_video_*.jpg'
train_images_paths = []
for path in glob(train_images_pattern):
    train_images_paths.append(path);

In [3]:
len(train_images_paths)

23287

In [4]:
train_images_paths[0:10]      # File names are not in "natural order"

['data/m2cai16-tool/train_dataset/images\\tool_video_01_0.jpg',
 'data/m2cai16-tool/train_dataset/images\\tool_video_01_100.jpg',
 'data/m2cai16-tool/train_dataset/images\\tool_video_01_1000.jpg',
 'data/m2cai16-tool/train_dataset/images\\tool_video_01_10000.jpg',
 'data/m2cai16-tool/train_dataset/images\\tool_video_01_10025.jpg',
 'data/m2cai16-tool/train_dataset/images\\tool_video_01_10050.jpg',
 'data/m2cai16-tool/train_dataset/images\\tool_video_01_10075.jpg',
 'data/m2cai16-tool/train_dataset/images\\tool_video_01_10100.jpg',
 'data/m2cai16-tool/train_dataset/images\\tool_video_01_10125.jpg',
 'data/m2cai16-tool/train_dataset/images\\tool_video_01_10150.jpg']

### Comparator for sorting the file paths in natural order

In [4]:
import re
def compare_key(s):
    r = r"tool_video_(\d*)_(\d*)"
    m = re.search(r, s)
    return int(m.group(1))*100000 + int(m.group(2))

In [6]:
train_images_paths.sort(key = compare_key)

In [17]:
train_images_paths[0:10]

['data/m2cai16-tool/train_dataset/images\\tool_video_01_0.jpg',
 'data/m2cai16-tool/train_dataset/images\\tool_video_01_25.jpg',
 'data/m2cai16-tool/train_dataset/images\\tool_video_01_50.jpg',
 'data/m2cai16-tool/train_dataset/images\\tool_video_01_75.jpg',
 'data/m2cai16-tool/train_dataset/images\\tool_video_01_100.jpg',
 'data/m2cai16-tool/train_dataset/images\\tool_video_01_125.jpg',
 'data/m2cai16-tool/train_dataset/images\\tool_video_01_150.jpg',
 'data/m2cai16-tool/train_dataset/images\\tool_video_01_175.jpg',
 'data/m2cai16-tool/train_dataset/images\\tool_video_01_200.jpg',
 'data/m2cai16-tool/train_dataset/images\\tool_video_01_225.jpg']

### Create H5 dataset

In [108]:
import h5py

# Data preperation reference.
# http://machinelearninguru.com/deep_learning/data_preparation/hdf5/hdf5.html

train_data_dump_path = 'data/train_dataset.h5'
train_data_dump = h5py.File(train_data_dump_path, mode='r+')

In [110]:
train_data_dump.close()

In [92]:
train_shape = (len(train_images_paths), 224, 224, 3)
train_data_dump.create_dataset("train_img", train_shape, np.int8)

<HDF5 dataset "train_img": shape (23287, 224, 224, 3), type "|i1">

In [93]:
# train_df.shape
train_data_dump.create_dataset("train_label", data=train_dfs)

<HDF5 dataset "train_label": shape (23287, 7), type "<i8">

In [109]:
list(train_data_dump.keys())

['train_img', 'train_label']

In [95]:
for i in tqdm(range(len(train_images_paths))):
        img = cv2.imread(train_images_paths[i])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (224, 224), interpolation = cv2.INTER_CUBIC)
        train_data_dump['train_img'][i, ...] = img[None]                        
            
            

100%|██████████████████████████████████████████████████████████████████████████████████████████| 23287/23287 [03:07<00:00, 124.06it/s]


In [96]:
train_data_dump.close()

## Extract test images

In [8]:
filepath = 'data/m2cai16-tool/test_dataset/*.mp4'
output_path = 'data/m2cai16-tool/test_dataset/images'

# Clean dir
if not os.path.exists(output_path):
    os.mkdir(output_path)
for file in glob(output_path+'/*'):
    os.remove(file)
    
for filename in glob(filepath):
    video2frame(filename, output_path)
    

data/m2cai16-tool/test_dataset\tool_video_11.mp4 tool_video_11 4410.0
data/m2cai16-tool/test_dataset\tool_video_12.mp4 tool_video_12 2033.0
data/m2cai16-tool/test_dataset\tool_video_13.mp4 tool_video_13 2401.0
data/m2cai16-tool/test_dataset\tool_video_14.mp4 tool_video_14 1874.0
data/m2cai16-tool/test_dataset\tool_video_15.mp4 tool_video_15 1825.0


In [9]:
test_dfs = []
test_label_path = 'data/m2cai16-tool/test_dataset/tool_video_??.txt'
for filepath in glob(test_label_path):    
    tdf = pd.read_table(filepath)
    print(filepath, tdf.shape)
    test_dfs.append(tdf)    
test_dfs = pd.concat(test_dfs)
test_dfs.drop('Frame', 1, inplace=True)
test_dfs.describe()

data/m2cai16-tool/test_dataset\tool_video_11.txt (4410, 8)
data/m2cai16-tool/test_dataset\tool_video_12.txt (2033, 8)
data/m2cai16-tool/test_dataset\tool_video_13.txt (2399, 8)
data/m2cai16-tool/test_dataset\tool_video_14.txt (1874, 8)
data/m2cai16-tool/test_dataset\tool_video_15.txt (1825, 8)


,Grasper,Bipolar,Hook,Scissors,Clipper,Irrigator,SpecimenBag
count,12541.000000,12541.000000,12541.000000,12541.000000,12541.000000,12541.000000,12541.000000
mean,0.523961,0.028626,0.594370,0.012599,0.025118,0.012678,0.038514
std,0.499445,0.166760,0.491033,0.111539,0.156489,0.111887,0.192440
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
test_dfs = np.array(test_dfs)

In [11]:
test_images_pattern = 'data/m2cai16-tool/test_dataset/images/tool_video_*.jpg'
test_images_paths = []
for path in glob(test_images_pattern):
    test_images_paths.append(path);

In [12]:
len(test_images_paths)

12541

In [13]:
test_images_paths[0:10]      # File names are not in "natural order"

['data/m2cai16-tool/test_dataset/images\\tool_video_11_0.jpg',
 'data/m2cai16-tool/test_dataset/images\\tool_video_11_100.jpg',
 'data/m2cai16-tool/test_dataset/images\\tool_video_11_1000.jpg',
 'data/m2cai16-tool/test_dataset/images\\tool_video_11_10000.jpg',
 'data/m2cai16-tool/test_dataset/images\\tool_video_11_100000.jpg',
 'data/m2cai16-tool/test_dataset/images\\tool_video_11_100025.jpg',
 'data/m2cai16-tool/test_dataset/images\\tool_video_11_100050.jpg',
 'data/m2cai16-tool/test_dataset/images\\tool_video_11_100075.jpg',
 'data/m2cai16-tool/test_dataset/images\\tool_video_11_100100.jpg',
 'data/m2cai16-tool/test_dataset/images\\tool_video_11_100125.jpg']

In [14]:
test_images_paths.sort(key = compare_key)

In [15]:
test_images_paths[0:10]

['data/m2cai16-tool/test_dataset/images\\tool_video_11_0.jpg',
 'data/m2cai16-tool/test_dataset/images\\tool_video_11_25.jpg',
 'data/m2cai16-tool/test_dataset/images\\tool_video_11_50.jpg',
 'data/m2cai16-tool/test_dataset/images\\tool_video_11_75.jpg',
 'data/m2cai16-tool/test_dataset/images\\tool_video_11_100.jpg',
 'data/m2cai16-tool/test_dataset/images\\tool_video_11_125.jpg',
 'data/m2cai16-tool/test_dataset/images\\tool_video_11_150.jpg',
 'data/m2cai16-tool/test_dataset/images\\tool_video_11_175.jpg',
 'data/m2cai16-tool/test_dataset/images\\tool_video_11_200.jpg',
 'data/m2cai16-tool/test_dataset/images\\tool_video_11_225.jpg']

In [16]:
test_data_dump_path = 'data/test_dataset.h5'
test_data_dump = h5py.File(test_data_dump_path, mode='w')

In [127]:
test_data_dump.close()

In [18]:
test_shape = (len(test_images_paths), 224, 224, 3)
test_data_dump.create_dataset("test_img", test_shape, np.int8)

<HDF5 dataset "test_img": shape (12541, 224, 224, 3), type "|i1">

In [19]:
# test_dfs.shape
test_data_dump.create_dataset("test_label", data=test_dfs)

<HDF5 dataset "test_label": shape (12541, 7), type "<i8">

In [24]:
list(test_data_dump.keys())

['test_img', 'test_label']

In [21]:
test_data_dump.file

<HDF5 file "test_dataset.h5" (mode r+)>

In [23]:
for i in tqdm(range(len(test_images_paths))):
        img = cv2.imread(test_images_paths[i])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (224, 224), interpolation = cv2.INTER_CUBIC)
        test_data_dump['test_img'][i, ...] = img[None]                        
            
            

100%|███████████████████████████████████████████████████████████████████████████████████████████| 12541/12541 [03:34<00:00, 58.56it/s]


In [ ]:
test_data_dump.close()